# Overview

**Inputs:**
 - Emissions or GMT scenarios from IAM.  
 - Climate impacts maps by Global Warming Level.  

**Outputs:**
 - Maps of climate impacts through time, per scenario.

Steps  
1. Import climate impacts database. NetCDF 4-d file with different impacts as variables.
2. Import the scenarios to be run. Can be global mean temp scenarios or Emissions|CO2 scenarios.
3. Preprocessing steps.
4. Run.


In [3]:
import dask
import glob
import numpy as np
import os
import pyam
import time
import xarray as xr
import yaml
from dask import delayed
from dask.distributed import Client
from process_config import *
from rime_functions import *

with open(yaml_path, "r") as f:
    params = yaml.full_load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\users\\byers\\Github\\climate_impacts_processing\\hotspots.yml'

### Set up Dask for monitoring Dashboard


In [4]:
dask.config.set(scheduler="processes")
dask.config.set(num_workers=num_workers)
client = Client()


Open http://localhost:8787/status

In [ ]:
### Import the climate impacts database files


### Import scenarios data

Load an IAMC scenarios dataset.  
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures.  


In [6]:
df_scens_in = pyam.IamDataFrame(fname_input_scenarios)
dft = df_scens_in.filter(variable=temp_variable)
dft = np.round(dft.as_pandas()[pyam.IAMC_IDX + ["year", "value", "Ssp_family"]], 2)
dft = pyam.IamDataFrame(dft)
# Replace & fill missing SSP scenario allocation
# dft = ssp_helper(dft, ssp_meta_col="Ssp_family", default_ssp="SSP2")


pyam.core - INFO: Reading file C:\Users\byers\IIASA\ECE.prog - Documents\Research Theme - NEXUS\Hotspots_Explorer_2p0\rcre_testing\testing_2\emissions_temp_AR6_small.xlsx
pyam.core - INFO: Reading meta indicators
pyam.logging - ERROR: Empty cells in `data` (columns: 'ssp_family'):
                       model                          scenario region  \
29240              IMAGE 3.2                     SSP2-baseline  World   
29241              IMAGE 3.2                     SSP2-baseline  World   
29242              IMAGE 3.2                     SSP2-baseline  World   
29243              IMAGE 3.2                     SSP2-baseline  World   
29244              IMAGE 3.2                     SSP2-baseline  World   
...                      ...                               ...    ...   
87543  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   
87544  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   
87545  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-media

ValueError: Empty cells in `data` (columns: 'ssp_family'):
                       model                          scenario region  \
29240              IMAGE 3.2                     SSP2-baseline  World   
29241              IMAGE 3.2                     SSP2-baseline  World   
29242              IMAGE 3.2                     SSP2-baseline  World   
29243              IMAGE 3.2                     SSP2-baseline  World   
29244              IMAGE 3.2                     SSP2-baseline  World   
...                      ...                               ...    ...   
87543  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   
87544  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   
87545  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   
87546  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   
87547  REMIND-MAgPIE 2.1-4.2  NGFS2_Net-Zero 2050 - IPD-median  World   

                                                variable unit  y

In [ ]:
dft = ssp_helper(dft, ssp_meta_col="Ssp_family", default_ssp="SSP2")


## Example multiple scenarios, 1 climate indicator

In [ ]:
print("Test multiple scenarios, 1 indicator")
start = time.time()

ind = "precip"
var = "sdii"
short = params["indicators"][ind][var]["short_name"]

ssp = "ssp2"



files = glob.glob(os.path.join(impact_data_dir, ind, f"*{short}_{ssp}*{ftype}.nc4"))
mapdata = xr.open_mfdataset(
    files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
)


map_out_MS = map_transform_gmt_multi_dask(
    dft.filter(model="POLES ADVANCE"),
    mapdata,
    years,
    use_dask=True,
    gmt_name="threshold",
    interpolation=interpolation,
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MS.data_vars}
filename = f"{output_folder_maps}scenario_maps_multiscenario_{ftype}.nc"
map_out_MS.to_netcdf(filename, encoding=encoding)

print("FINISHED Test multiple scenarios, 1 indicator")
print(f"{time.time()-start}")